In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_data = 5000   # number of data
num_vars = 5      # number of decision variables
num_ints = 5      # number of integer decision variables
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_train = np.random.uniform(1.0, 6.0, (train_size, 1)).astype(np.float32)
a_train = np.random.uniform(0.2, 1.2, (train_size, num_vars-1)).astype(np.float32)
p_test = np.random.uniform(1.0, 6.0, (test_size, 1)).astype(np.float32)
a_test = np.random.uniform(0.2, 1.2, (test_size, num_vars-1)).astype(np.float32)
p_dev = np.random.uniform(1.0, 6.0, (val_size, 1)).astype(np.float32)
a_dev = np.random.uniform(0.2, 1.2, (val_size, num_vars-1)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_test}, name="dev")

In [6]:
# torch dataloaders
from torch.utils.data import DataLoader
loader_train = DataLoader(data_train, batch_size=32, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size=32, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size=32, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## NM Problem

In [7]:
import neuromancer as nm
from problem.neural import probRosenbrock

def getNMProb(round_module):
    # parameters
    p = nm.constraint.variable("p")
    a = nm.constraint.variable("a")
    # variables
    x_bar = nm.constraint.variable("x_bar")
    x_rnd = nm.constraint.variable("x_rnd")

    # model
    obj_bar, constrs_bar = probRosenbrock(x_bar, p, a, num_vars=num_vars, alpha=100)
    obj_rnd, constrs_rnd = probRosenbrock(x_rnd, p, a, num_vars=num_vars, alpha=100)

    # define neural architecture for the solution mapping
    func = nm.modules.blocks.MLP(insize=num_vars, outsize=num_vars, bias=True,
                                 linear_map=nm.slim.maps["linear"], nonlin=nn.ReLU, hsizes=[80]*4)
    # solution map from model parameters: sol_map(p) -> x
    sol_map = nm.system.Node(func, ["p", "a"], ["x_bar"], name="smap")

    # penalty loss for mapping
    components = [sol_map]
    loss = nm.loss.PenaltyLoss(obj_bar, constrs_bar)
    problem = nm.problem.Problem(components, loss)

    # penalty loss for rounding
    components = [sol_map, round_module]
    loss = nm.loss.PenaltyLoss(obj_rnd, constrs_rnd)
    problem_rnd = nm.problem.Problem(components, loss)

    return problem, problem_rnd

## Exact Solver

In [8]:
from problem.solver import exactRosenbrock
model = exactRosenbrock(n_vars=num_vars, n_integers=num_ints)

In [9]:
sols, objvals, conviols, elapseds = [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    model.setParamValue(p, *a)
    tick = time.time()
    xval, objval = model.solve("scip")
    tock = time.time()
    sols.append(list(xval.values()))
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:37<00:00, 10.21it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean      1.786264               0.0      0.097387
std       1.361469               0.0      0.034884
min       0.000000               0.0      0.062039
25%       0.466428               0.0      0.076015
50%       1.731629               0.0      0.089871
75%       2.953879               0.0      0.092289
max       4.199299               0.0      0.244033


In [10]:
df

,Sol,Obj Val,Constraints Viol,Elapsed Time
0,"[1.0, 0.0, 1.0, 1.0, 0.0]",1.634025,0.0,0.131417
1,"[1.0, 1.0, 1.0, 1.0, 0.0]",0.261138,0.0,0.091855
2,"[1.0, 1.0, 1.0, 0.0, 0.0]",1.244660,0.0,0.076746
3,"[1.0, 0.0, 0.0, 0.0, 0.0]",3.591436,0.0,0.074814
4,"[1.0, 0.0, 0.0, 0.0, 0.0]",4.187316,0.0,0.078243
...,...,...,...,...
995,"[1.0, 0.0, 0.0, 0.0, 0.0]",4.089960,0.0,0.075583
996,"[1.0, 1.0, 1.0, 0.0, 0.0]",1.625817,0.0,0.076519
997,"[1.0, 1.0, 1.0, 1.0, 0.0]",0.959324,0.0,0.106097
998,"[1.0, 1.0, 1.0, 1.0, 0.0]",1.077039,0.0,0.088842


## Heuristic

In [11]:
from heuristic import naive_round

In [12]:
# relaxed model
model_rel = model.relax()

In [13]:
sols, objvals, conviols, elapseds = [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    model_rel.setParamValue(p, *a)
    tick = time.time()
    xval_init, _ = model_rel.solve("scip", max_iter=100)
    naive_round(xval_init, model)
    tock = time.time()
    xval, objval = model.getVal()
    sols.append(list(xval.values()))
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [18:34<00:00,  1.11s/it]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean      0.346535          0.014486      1.113325
std       0.572597          0.104144      0.357769
min       0.000000          0.000000      0.432500
25%       0.000000          0.000000      0.881712
50%       0.000000          0.000000      1.009262
75%       0.372265          0.000000      1.320632
max       2.503698          0.762442      2.361451


## Learning to Round

In [14]:
from model.layer import netFC
from model.round import roundModel
# round x
layers_rnd = netFC(input_dim=num_vars*2, hidden_dims=[80]*4, output_dim=num_vars)
round_func = roundModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x_bar"], output_keys=["x_rnd"],
                        int_ind={"x_bar":model.intInd}, name="round")
_, problem = getNMProb(round_func)

In [15]:
# training
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 49.70849609375
epoch: 1  train_loss: 41.51247024536133
epoch: 2  train_loss: 40.15127944946289
epoch: 3  train_loss: 50.554386138916016
epoch: 4  train_loss: 45.82106018066406
epoch: 5  train_loss: 49.82112121582031
epoch: 6  train_loss: 40.62741470336914
epoch: 7  train_loss: 39.87377166748047
epoch: 8  train_loss: 47.59299087524414
epoch: 9  train_loss: 42.070716857910156
epoch: 10  train_loss: 41.75148010253906
epoch: 11  train_loss: 37.79279327392578
epoch: 12  train_loss: 46.500038146972656
epoch: 13  train_loss: 39.58802032470703
epoch: 14  train_loss: 45.82360076904297
epoch: 15  train_loss: 49.97172546386719
epoch: 16  train_loss: 31.36939239501953
epoch: 17  train_loss: 40.5133171081543
epoch: 18  train_loss: 38.941890716552734
epoch: 19  train_loss: 26.786985397338867
epoch: 20  train_loss: 28.458574295043945
epoch: 21  train_loss: 25.684894561767578
epoch: 22  train_loss: 28.383136749267578
epoch: 23  train_loss: 26.57742691040039
epoch: 24  train_loss:

In [16]:
sols, objvals, conviols, elapseds = [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), "a":torch.tensor(np.array([a]), dtype=torch.float32), "name": "test"}
    tick = time.time()
    output = problem(datapoints)
    tock = time.time()
    x = output["test_x_rnd"]
    # get values
    for ind in model.x:
        model.x[ind].value = x[0, ind].item()
    xval, objval = model.getVal()
    sols.append(xval.values())
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 184.06it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean      3.580870          0.704800      0.004821
std       0.655242          0.857373      0.001216
min       1.750512          0.000000      0.002003
25%       3.246814          0.000000      0.003999
50%       3.766097          0.000000      0.004556
75%       3.766097          1.762442      0.005614
max       4.519283          2.762442      0.010525


## Learnable Threshold

In [17]:
from model.layer import netFC
from model.threshold import roundThresholdModel
# round x
layers_rnd = netFC(input_dim=num_vars*2, hidden_dims=[80]*4, output_dim=num_vars)
round_func = roundThresholdModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x_bar"], output_keys=["x_rnd"],
                                 int_ind={"x_bar":model.intInd}, name="round")
_, problem = getNMProb(round_func)

In [18]:
# training
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 59.47038650512695
epoch: 1  train_loss: 46.17607879638672
epoch: 2  train_loss: 42.86919021606445
epoch: 3  train_loss: 44.91936111450195
epoch: 4  train_loss: 40.279144287109375
epoch: 5  train_loss: 37.545692443847656
epoch: 6  train_loss: 43.543426513671875
epoch: 7  train_loss: 42.39023208618164
epoch: 8  train_loss: 46.024139404296875
epoch: 9  train_loss: 59.60663604736328
epoch: 10  train_loss: 59.56098937988281
epoch: 11  train_loss: 49.43817138671875
epoch: 12  train_loss: 62.17286682128906
epoch: 13  train_loss: 72.67794036865234
epoch: 14  train_loss: 59.97739028930664
epoch: 15  train_loss: 63.3626823425293
epoch: 16  train_loss: 58.953956604003906
epoch: 17  train_loss: 67.74063873291016
epoch: 18  train_loss: 83.29310607910156
epoch: 19  train_loss: 69.541015625
epoch: 20  train_loss: 65.42090606689453
epoch: 21  train_loss: 72.77654266357422
epoch: 22  train_loss: 75.35345458984375
epoch: 23  train_loss: 80.69808959960938
epoch: 24  train_loss: 72.9

In [19]:
sols, objvals, conviols, elapseds = [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), "a":torch.tensor(np.array([a]), dtype=torch.float32), "name": "test"}
    tick = time.time()
    output = problem(datapoints)
    tock = time.time()
    x = output["test_x_rnd"]
    # get values
    for ind in model.x:
        model.x[ind].value = x[0, ind].item()
    xval, objval = model.getVal()
    sols.append(xval.values())
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 187.16it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean      6.831584          1.355442      0.004681
std       1.731548          0.497593      0.001181
min       3.246814          0.762442      0.002547
25%       7.262399          0.762442      0.003999
50%       7.262399          1.762442      0.004524
75%       8.015585          1.762442      0.005275
max       8.015585          2.762442      0.018537


## Learning to Round with Fixed Solution Mapping

In [20]:
from model.layer import netFC
from model.round import roundModel
# round x
layers_rnd = netFC(input_dim=num_vars*2, hidden_dims=[80]*4, output_dim=num_vars)
round_func = roundModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x_bar"], output_keys=["x_rnd"],
                        int_ind={"x_bar":model.intInd}, name="round")
problem_rel, problem_rnd = getNMProb(round_func)

In [21]:
# training for mapping
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem_rel.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem_rel, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 33.30405044555664
epoch: 1  train_loss: 0.7042540907859802
epoch: 2  train_loss: 0.44580766558647156
epoch: 3  train_loss: 0.41395464539527893
epoch: 4  train_loss: 0.4325869083404541
epoch: 5  train_loss: 0.43818360567092896
epoch: 6  train_loss: 0.4464978277683258
epoch: 7  train_loss: 0.42415156960487366
epoch: 8  train_loss: 0.3839186429977417
epoch: 9  train_loss: 0.37431758642196655
epoch: 10  train_loss: 0.4232294261455536
epoch: 11  train_loss: 0.38242509961128235
epoch: 12  train_loss: 0.3845195770263672
epoch: 13  train_loss: 0.3933694660663605
epoch: 14  train_loss: 0.3860446810722351
epoch: 15  train_loss: 0.3651963174343109
epoch: 16  train_loss: 0.37582266330718994
epoch: 17  train_loss: 0.40678274631500244
epoch: 18  train_loss: 0.4135703146457672
epoch: 19  train_loss: 0.356105238199234
epoch: 20  train_loss: 0.40241971611976624
epoch: 21  train_loss: 0.368284672498703
epoch: 22  train_loss: 0.37881582975387573
epoch: 23  train_loss: 0.365775823593

In [22]:
# freeze sol mapping
#problem_rel.freeze()
# training for rounding
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem_rnd, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 131.53704833984375
epoch: 1  train_loss: 129.52284240722656
epoch: 2  train_loss: 133.23451232910156
epoch: 3  train_loss: 133.18605041503906
epoch: 4  train_loss: 130.60865783691406
epoch: 5  train_loss: 133.2777557373047
epoch: 6  train_loss: 133.48606872558594
epoch: 7  train_loss: 129.01168823242188
epoch: 8  train_loss: 126.90194702148438
epoch: 9  train_loss: 129.93064880371094
epoch: 10  train_loss: 131.24891662597656
epoch: 11  train_loss: 129.44924926757812
epoch: 12  train_loss: 133.0521697998047
epoch: 13  train_loss: 131.92398071289062
epoch: 14  train_loss: 133.05322265625
epoch: 15  train_loss: 128.50523376464844
epoch: 16  train_loss: 132.8893585205078
epoch: 17  train_loss: 131.83924865722656
epoch: 18  train_loss: 131.30224609375
epoch: 19  train_loss: 130.4916229248047
epoch: 20  train_loss: 132.34283447265625
epoch: 21  train_loss: 132.94374084472656
epoch: 22  train_loss: 129.9585723876953
epoch: 23  train_loss: 135.364990234375
epoch: 24  trai

In [23]:
sols, objvals, conviols, elapseds = [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), "a":torch.tensor(np.array([a]), dtype=torch.float32), "name": "test"}
    tick = time.time()
    output = problem_rnd(datapoints)
    tock = time.time()
    x = output["test_x_rnd"]
    # get values
    for ind in model.x:
        model.x[ind].value = x[0, ind].item()
    xval, objval = model.getVal()
    sols.append(xval.values())
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 184.70it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean      4.132830          1.526321      0.004759
std       2.375297          1.165372      0.001110
min       0.000000          0.000000      0.002683
25%       3.262399          0.762442      0.004000
50%       4.000000          1.475116      0.004543
75%       4.147018          2.762442      0.005506
max      16.390409          8.475116      0.009948


### Learning to Feasibility Pump

In [24]:
from model.layer import netFC
# define neural architecture for the solution mapping
sol_func = nm.modules.blocks.MLP(insize=num_vars, outsize=num_vars, bias=True,
                                 linear_map=nm.slim.maps["linear"], nonlin=nn.ReLU,
                                 hsizes=[80]*4)
# define neural architecture for rounding
rnd_layer = netFC(input_dim=num_vars*2, hidden_dims=[80]*4, output_dim=num_vars)

In [25]:
# function to get nm optimization model
from problem.neural import probRosenbrock
def getProb(x, p, a):
    return probRosenbrock(x, p, a, num_vars=num_vars, alpha=100)

In [26]:
from model import feasibilityPumpModel
# feasibility pump model
problem_rel, problem_fp = feasibilityPumpModel(["p", "a"], getProb, sol_func, rnd_layer, int_ind=model.intInd, num_iters=3)

In [27]:
# training for mapping
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem_rel.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem_rel, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 31.091703414916992
epoch: 1  train_loss: 0.5559331774711609
epoch: 2  train_loss: 0.4705972969532013
epoch: 3  train_loss: 0.45412054657936096
epoch: 4  train_loss: 0.4110414981842041
epoch: 5  train_loss: 0.40120071172714233
epoch: 6  train_loss: 0.42559176683425903
epoch: 7  train_loss: 0.42336714267730713
epoch: 8  train_loss: 0.3654492199420929
epoch: 9  train_loss: 0.36057934165000916
epoch: 10  train_loss: 0.3620113134384155
epoch: 11  train_loss: 0.4111559987068176
epoch: 12  train_loss: 0.42911890149116516
epoch: 13  train_loss: 0.4039905369281769
epoch: 14  train_loss: 0.391169011592865
epoch: 15  train_loss: 0.38851964473724365
epoch: 16  train_loss: 0.4309215247631073
epoch: 17  train_loss: 0.36927974224090576
epoch: 18  train_loss: 0.33142414689064026
epoch: 19  train_loss: 0.41081157326698303
epoch: 20  train_loss: 0.38458412885665894
epoch: 21  train_loss: 0.40109512209892273
epoch: 22  train_loss: 0.3996732532978058
epoch: 23  train_loss: 0.39662161

In [28]:
# freeze sol mapping
#problem_rel.freeze()
# training for feasibility pump
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem_fp, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 1209.739990234375
epoch: 1  train_loss: 1228.4407958984375
epoch: 2  train_loss: 1232.14013671875
epoch: 3  train_loss: 1232.2135009765625
epoch: 4  train_loss: 1245.966552734375
epoch: 5  train_loss: 1247.1795654296875
epoch: 6  train_loss: 1243.58154296875
epoch: 7  train_loss: 1229.1455078125
epoch: 8  train_loss: 1245.62939453125
epoch: 9  train_loss: 1240.62353515625
epoch: 10  train_loss: 1243.5201416015625
epoch: 11  train_loss: 1233.32275390625
epoch: 12  train_loss: 1256.79052734375
epoch: 13  train_loss: 1238.919189453125
epoch: 14  train_loss: 1233.199951171875
epoch: 15  train_loss: 1242.6917724609375
epoch: 16  train_loss: 1235.55419921875
epoch: 17  train_loss: 1234.8634033203125
epoch: 18  train_loss: 1246.98388671875
epoch: 19  train_loss: 1227.5555419921875
epoch: 20  train_loss: 1229.7686767578125
epoch: 21  train_loss: 1232.7969970703125
epoch: 22  train_loss: 1232.2926025390625
epoch: 23  train_loss: 1253.1356201171875
epoch: 24  train_loss: 12

In [29]:
sols, objvals, conviols, elapseds = [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), "a":torch.tensor(np.array([a]), dtype=torch.float32), "name": "test"}
    tick = time.time()
    output = problem(datapoints)
    tock = time.time()
    x = output["test_x_rnd"]
    # get values
    for ind in model.x:
        model.x[ind].value = x[0, ind].item()
    xval, objval = model.getVal()
    sols.append(xval.values())
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 179.45it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean      6.831584          1.355442      0.004911
std       1.731548          0.497593      0.001176
min       3.246814          0.762442      0.002025
25%       7.262399          0.762442      0.004014
50%       7.262399          1.762442      0.004664
75%       8.015585          1.762442      0.005669
max       8.015585          2.762442      0.010804
